In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from operator import add
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("adam-axelsson-part-b")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.cores.max",4)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [ ]:
#B.1
parking = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.119:9000/parking-citations.csv')\
    .cache()

parking.show()


In [ ]:
#B.2

parking.printSchema()


In [ ]:
#B.3
print("Number of rows: %d \n" % parking.count())

#B.4
print("Number of partitions: %d" % parking.rdd.getNumPartitions())

In [ ]:
#B.5
parking = parking.drop(*("VIN","Latitude", "Longitude"))

#B.6
from pyspark.sql.functions import col
parking = parking.withColumn("Fine amount", col("Fine amount").cast('float'))
max_fine = parking.agg({'Fine amount': 'max'}).collect()[0]
print("Maximum fine: %d \n" % max_fine[0])
all_max_fines = parking.where(col("Fine amount") == max_fine[0])
print("Number of fines for %d: %d \n " % (max_fine[0], all_max_fines.count()))

In [ ]:
#B.7
count_make = parking.groupBy("Make").count()
count_make_sorted =  count_make.orderBy("count",ascending=False)
top_20_make = count_make_sorted.limit(20)
top_20_make.show()

In [ ]:
#B.8
from pyspark.sql.functions import udf
from pyspark.sql.types import *
def color_long_column(column):
    COLORS = { 
        'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black', 
        'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze', 
        'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold', 
        'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory', 
        'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon', 
        'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver', 
        'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White', 
        'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown' 
        }
    if color in COLORS.keys():
        return COLORS.get(color)
    else: 
        return color
    
udf_color_long_column = udf(color_long_column, StringType())
color_df = parking.withColumn("color long", udf_color_long_column("color"))
color_df.show()
        
parking.withColumn(col("long color"), None).show()
#parking.withColumn("color long", color_long_column(col("color"))).show()